In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('iris.csv')

In [7]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [8]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [10]:
#column names can not contain white spaces and other sympols like parentheses etc
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']

In [11]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target'], dtype='object')

In [12]:
#numbers in target column have to be integers
df["target"] = df["target"].apply(int)

In [13]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [14]:
#TRAIN TEST SPLIT

In [15]:
from sklearn.model_selection import train_test_split


In [23]:
X = df.drop("target", axis = 1)

In [24]:
y = df["target"]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [26]:
#Let's SEE how to use the simpler Estimator interface

In [27]:
import tensorflow as tf

D:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [28]:
#CREATE FEATURE COLUMNS

In [29]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [30]:
feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [31]:
feat_cols

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [32]:
#INPUT FUNCTION

In [405]:
# there is also a pandas_input_fn we'll see in the exercise!!
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=30,num_epochs=5,shuffle=True)

In [406]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10, 10, 10, 10], n_classes=3,feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\AppData\\Local\\Temp\\tmpxl1ygtih', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016DDBB63CF8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [407]:
#Use the predict method from the classifier model to CREATE PREDICTIONS from X_test

In [408]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [409]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Could not find trained model in model_dir: D:\AppData\Local\Temp\tmpxl1ygtih, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [410]:
note_predictions[0]

{'class_ids': array([1], dtype=int64),
 'classes': array([b'1'], dtype=object),
 'logits': array([0.34345472, 0.45407557, 0.05113225], dtype=float32),
 'probabilities': array([0.34922308, 0.39007217, 0.26070473], dtype=float32)}

In [411]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [412]:
#Now create a classification report and a Confusion Matrix.

In [413]:
from sklearn.metrics import classification_report,confusion_matrix

In [414]:
print(confusion_matrix(y_test,final_preds))

[[11  1  0]
 [ 0 14  0]
 [ 0 19  0]]


In [415]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       1.00      0.92      0.96        12
          1       0.41      1.00      0.58        14
          2       0.00      0.00      0.00        19

avg / total       0.39      0.56      0.44        45



D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
